In [4]:
# import requests
# from bs4 import BeautifulSoup
# import pandas as pd
# import time
# from tqdm import tqdm

# # Load the updated Excel file
# file_path = 'List_Perusahaan_With_Search_Terms.xlsx'
# data = pd.read_excel(file_path)

# # Your ScraperAPI key
# scraperapi_key = "04770e3a64ab57bfbbcbab3057042d72"

# # Function to perform Google search and filter for OJK links using ScraperAPI
# def get_ojk_link(search_term):
#     params = {
#         "api_key": scraperapi_key,
#         "url": f"https://www.google.com/search?q={search_term}"
#     }
#     try:
#         response = requests.get("http://api.scraperapi.com", params=params)
#         response.raise_for_status()
#         soup = BeautifulSoup(response.text, 'html.parser')
#         for link in soup.find_all('a', href=True):
#             href = link['href']
#             if "https://ojk.go.id/id/kanal/pasar-modal/" in href or "https://www.ojk.go.id/id/kanal/pasar-modal/" in href:
#                 return href 
#         time.sleep(2)  # Sleep for 2 seconds to avoid being blocked
#     except requests.exceptions.HTTPError as http_err:
#         print(f"HTTP error occurred during search for term '{search_term}': {http_err}")
#     except Exception as err:
#         print(f"Error during search for term '{search_term}': {err}")
#     return "No link found"

# # Initialize list to hold results
# ojk_links_list = []
# ## row 1 = start_row 0
# # Set start row (0-based index, so row 2 is index 1)
# start_row = 750

# # Iterate over the rows starting from the specified start row
# save_interval = 50  # Save the file every 50 rows

# for i, (index, row) in enumerate(tqdm(data.iloc[start_row:].iterrows(), total=data.shape[0] - start_row, desc="Scraping OJK Links")):
#     search_term = row['Search_Term']
#     ojk_link = get_ojk_link(search_term)
#     ojk_links_list.append(ojk_link)
    
#     # Save results to Excel at specified intervals
#     if (i + 1) % save_interval == 0:
#         save_start = start_row + i - save_interval + 1
#         save_end = start_row + i
#         data.loc[save_start:save_end, 'OJK_Links'] = ojk_links_list
#         # Save DataFrame to Excel
#         output_path = f'Output_{start_row + i + 1}.xlsx'
#         data.to_excel(output_path, index=False)
#         print(f"Saved interim results to {output_path}")
#         ojk_links_list = []  # Reset the list after saving

# # Save any remaining results
# if ojk_links_list:
#     save_start = start_row + i - len(ojk_links_list) + 1
#     save_end = start_row + i
#     data.loc[save_start:save_end, 'OJK_Links'] = ojk_links_list
#     output_path = 'List_Perusahaan_With_OJK_Links_final.xlsx'
#     data.to_excel(output_path, index=False)
#     print(f"Scraping selesai dan hasilnya disimpan di {output_path}")

Scraping OJK Links:  32%|███▏      | 50/155 [07:51<19:40, 11.24s/it]

Saved interim results to Output_800.xlsx


Scraping OJK Links:  65%|██████▍   | 100/155 [14:34<11:42, 12.77s/it]

Saved interim results to Output_850.xlsx


Scraping OJK Links:  97%|█████████▋| 150/155 [22:48<00:37,  7.42s/it]

Saved interim results to Output_900.xlsx


Scraping OJK Links: 100%|██████████| 155/155 [23:31<00:00,  9.11s/it]

Scraping selesai dan hasilnya disimpan di List_Perusahaan_With_OJK_Links_final.xlsx


In [3]:
import pandas as pd
import requests
import os
import re

# Load the dataset
file_path = 'List_Perusahaan_With_OJK_Links_final.xlsx'  # Ganti dengan path yang sesuai
df = pd.read_excel(file_path)

# Function to check if a link contains "/Documents/Pages/"
def is_excluded(link):
    return re.search(r"/Documents/Pages/", link) is not None

# Function to check if a link is valid (starts with http or https)
def is_valid_url(link):
    return link.startswith('http')

# Filter out the links that contain "/Documents/Pages/" and keep only valid links
filtered_links = df['OJK_Links'].apply(lambda x: x if not is_excluded(x) else None).dropna()
valid_links = [(index, row.NAME, link) for index, row, link in zip(df.index, df.itertuples(), filtered_links) if is_valid_url(link)]

# Directory to save the PDFs
save_dir = 'pdf'  # Ganti dengan path yang diinginkan

# Create directory if it doesn't exist
os.makedirs(save_dir, exist_ok=True)

# Function to download a PDF
def download_pdf(url, save_path):
    response = requests.get(url)
    if response.status_code == 200:
        with open(save_path, 'wb') as f:
            f.write(response.content)
        return True
    return False

# Download PDFs
for index, name, link in valid_links:
    pdf_name = f'{index}_{name}.pdf'
    save_path = os.path.join(save_dir, pdf_name)
    success = download_pdf(link, save_path)
    if success:
        print(f"Downloaded: {pdf_name}")
    else:
        print(f"Failed to download: {link}")


Downloaded: 2_Rukun Raharja Tbk.pdf
Downloaded: 3_Indo Kordsa Tbk.pdf
Downloaded: 9_PT Akbar Indo Makmur Stimec Tbk.pdf
Downloaded: 12_Inter-Delta Tbk.pdf
Downloaded: 14_Fortune Indonesia Tbk.pdf
Downloaded: 15_Danasupra Erapacific Tbk.pdf
Downloaded: 17_PT Wira Global Solusi Tbk.pdf
Downloaded: 24_PT Oscar Mitra Sukses Sejahtera Tbk.pdf
Downloaded: 25_PT Wahana Inti Makmur Tbk.pdf
Downloaded: 32_PT Imago Mulia Persada Tbk.pdf
Downloaded: 37_PT Andalan Sakti Primaindo Tbk.pdf
Downloaded: 41_Garuda Indonesia (Persero) Tbk.pdf
Downloaded: 47_Pembangunan Graha Lestari Indah Tbk.pdf
Downloaded: 49_PT Meta Epsi Tbk.pdf
Downloaded: 51_PT OBM Drilchem Tbk.pdf
Downloaded: 52_PT Jasnita Telekomindo Tbk.pdf
Downloaded: 58_PT Magna Investama Mandiri Tbk.pdf
Downloaded: 66_PT Dewata Freightinternational Tbk.pdf
Downloaded: 67_PT Fuji Finance Indonesia Tbk.pdf
Downloaded: 75_PT Asia Sejahtera Mina Tbk.pdf
Downloaded: 76_PT Lancartama Sejati Tbk.pdf
Downloaded: 80_PT Singaraja Putra Tbk.pdf
Download

In [4]:
import pandas as pd
import requests
import os
import re

# Load the dataset
file_path = 'List_Perusahaan_With_OJK_Links_final.xlsx'  # Ganti dengan path yang sesuai
df = pd.read_excel(file_path)

# Function to check if a link contains "/Documents/Pages/"
def is_excluded(link):
    return re.search(r"/Documents/Pages/", link) is not None

# Function to check if a link is valid (starts with http or https)
def is_valid_url(link):
    return link.startswith('http')

# Filter out the links that contain "/Documents/Pages/" and keep only valid links
filtered_links = df['OJK_Links'].apply(lambda x: x if not is_excluded(x) else None).dropna()

# List to store valid links along with their index and name
valid_links = []

# Iterate over the DataFrame rows starting from index 827
for index, row in df.iloc[827:].iterrows():
    link = row['OJK_Links']
    if not is_excluded(link) and is_valid_url(link):
        valid_links.append((index, row['NAME'], link))

# Check the number of valid links found
print(f"Total valid links found: {len(valid_links)}")

# Directory to save the PDFs
save_dir = 'pdf2'  # Ganti dengan path yang diinginkan

# Create directory if it doesn't exist
os.makedirs(save_dir, exist_ok=True)

# Function to download a PDF
def download_pdf(url, save_path):
    response = requests.get(url)
    if response.status_code == 200:
        with open(save_path, 'wb') as f:
            f.write(response.content)
        return True
    return False

# Download PDFs
for index, name, link in valid_links:
    pdf_name = f'{index}_{name}.pdf'.replace('/', '_')
    save_path = os.path.join(save_dir, pdf_name)
    success = download_pdf(link, save_path)
    if success:
        print(f"Downloaded: {pdf_name}")
    else:
        print(f"Failed to download: {link}")


Total valid links found: 17
Downloaded: 829_PT_Mitra_Komunikasi_Nusantara_Tbk.pdf
Downloaded: 836_Capitalinc_Investment_Tbk.pdf
Downloaded: 838_Metro_Realty_Tbk.pdf
Downloaded: 840_Hanson_International_Tbk.pdf
Downloaded: 844_Nipress_Tbk.pdf
Downloaded: 855_Polaris_Investama_Tbk.pdf
Downloaded: 857_Pool_Advista_Indonesia_Tbk.pdf
Downloaded: 868_Rig_Tenders_Tbk.pdf
Downloaded: 869_PT_Rimo_International_Lestari_Tbk.pdf
Downloaded: 870_Bentoel_International_Investama_Tbk.pdf
Downloaded: 874_Siwani_Makmur_Tbk.pdf
Downloaded: 879_Sugih_Energy_Tbk.pdf
Downloaded: 885_Tira_Austenite_Tbk.pdf
Downloaded: 889_Triwira_Insanlestari_Tbk.pdf
Downloaded: 893_Unggul_Indah_Cahaya_Tbk.pdf
Downloaded: 894_Nusantara_Inti_Corpora_Tbk.pdf
Downloaded: 897_PT_Victoria_Investama_Tbk.pdf


In [1]:
import pandas as pd
import requests
import os
import re

# Load the dataset
file_path = 'List_Perusahaan_With_OJK_Links_final.xlsx'  # Ganti dengan path yang sesuai
df = pd.read_excel(file_path)

# Function to check if a link contains "/Documents/Pages/"
def is_excluded(link):
    return re.search(r"/Documents/Pages/", link) is not None

# Function to check if a link is valid (starts with http or https)
def is_valid_url(link):
    return link.startswith('http')

# Filter out the links that contain "/Documents/Pages/" and keep only valid links
filtered_links = df['OJK_Links'].apply(lambda x: x if not is_excluded(x) else None).dropna()

# List to store valid links along with their index and name
valid_links = []

# Iterate over the DataFrame rows
for index, row in df.iterrows():
    link = row['OJK_Links']
    if not is_excluded(link) and is_valid_url(link):
        valid_links.append((index, row['NAME'], link))

# Check the number of valid links found
print(f"Total valid links found: {len(valid_links)}")

# Directory to save the PDFs
save_dir = 'pdf2'  # Ganti dengan path yang diinginkan

# Create directory if it doesn't exist
os.makedirs(save_dir, exist_ok=True)

# Function to download a PDF
def download_pdf(url, save_path):
    response = requests.get(url)
    if response.status_code == 200:
        with open(save_path, 'wb') as f:
            f.write(response.content)
        return True
    return False

# Download PDFs
for index, name, link in valid_links:
    pdf_name = f'{index}_{name}.pdf'.replace('/', '_')
    save_path = os.path.join(save_dir, pdf_name)
    success = download_pdf(link, save_path)
    if success:
        print(f"Downloaded: {pdf_name}")
    else:
        print(f"Failed to download: {link}")


Total valid links found: 420
Downloaded: 2_Rukun Raharja Tbk.pdf
Downloaded: 3_Indo Kordsa Tbk.pdf
Downloaded: 9_PT Akbar Indo Makmur Stimec Tbk.pdf
Downloaded: 12_Inter-Delta Tbk.pdf
Downloaded: 14_Fortune Indonesia Tbk.pdf
Downloaded: 15_Danasupra Erapacific Tbk.pdf
Downloaded: 18_Tanah Laut Tbk.pdf
Downloaded: 27_Perdana Karya Perkasa Tbk.pdf
Downloaded: 28_Express Transindo Utama Tbk.pdf
Downloaded: 35_Trikomsel Oke Tbk.pdf
Downloaded: 41_Garuda Indonesia (Persero) Tbk.pdf
Downloaded: 47_Pembangunan Graha Lestari Indah Tbk.pdf
Downloaded: 54_Lionmesh Prima Tbk.pdf
Downloaded: 56_Bekasi Asri Pemula Tbk.pdf
Downloaded: 58_PT Magna Investama Mandiri Tbk.pdf
Downloaded: 59_Perdana Bangun Pusaka Tbk.pdf
Downloaded: 65_Jakarta Kyoei Steel Works Tbk.pdf
Downloaded: 73_Kedaung Indah Can Tbk.pdf
Downloaded: 74_PT Harapan Duta Pertiwi Tbk.pdf
Downloaded: 82_PT Cilacap Samudera Fishing Industry Tbk.pdf
Downloaded: 83_PT Eka Sari Lorena Transport Tbk.pdf
Downloaded: 87_ICTSI JASA PRIMA Tbk.pdf